In [17]:
import sqlite3
import pandas as pd
import datetime

Data used for this project was downloaded from [here](https://pub.phyks.me/datasets/velib/2020/)

In [18]:
def convertSql(filepath, table):
    ary = []
    connection = sqlite3.connect(":memory:")
    cursor = connection.cursor()
    
    sql_file = open(filepath)
    sql_as_string = sql_file.read()
    cursor.executescript(sql_as_string)
    
    for row in cursor.execute("SELECT * FROM " + table):
        ary.append(row)
    
    return ary

Create dataframe with pandas from converted array

In [19]:
def createDF_activity(ary):
    return pd.DataFrame(ary, columns =['station_id', 'available_bikes', 'available_ebikes', 'free_stands', 'status', 'updated']) 

In [20]:
entries_week_preLockdown = convertSql("./data/week_11.sql", "stationsstats")
entries_week_lockdown = convertSql("./data/week_12.sql", "stationsstats")

stations_velib = convertSql("./data/week_11.sql", "stations")

In [21]:
df_activity = createDF_activity(entries_week_preLockdown + entries_week_lockdown)

In [22]:
df_stations = pd.DataFrame(stations_velib, columns =['station_id', 'name', 'address', 'latitude', 'longitude', 'banking','bonus','bike_stands'])
df_stations = df_stations.drop(columns=['address'])

### Get arrondissement from Geocodes 

In [23]:
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [24]:
def createPostcode(row):
    geolocator = Nominatim(user_agent="application")
    reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)
    location = reverse((row.latitude, row.longitude), language='en', exactly_one=True)
    try:
        return location.raw['address']['postcode']
    except:
        suburb = location.raw['address']['suburb']
        if "th" in suburb:
            length = len(suburb.split('th')[0])
            if length == 1:
                return "7500" + suburb.split('th')[0]
            else:
                return "750" + suburb.split('th')[0]
        elif "st" in suburb:
            return "7500" + suburb.split('st')[0]
        else:
            return "nan"

In [25]:
df_activity['updated'] = df_activity['updated'].apply(lambda x: datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))


In [26]:
df_activity = df_activity.drop_duplicates()
df_stations = df_stations.drop_duplicates(subset='station_id')

In [27]:
df_stations['postcode'] = df_stations.apply(lambda row: createPostcode(row), axis=1)

In [28]:
df_activity.head()

,station_id,available_bikes,available_ebikes,free_stands,status,updated
0,4020,6,0,15,Operative,2020-03-09 00:00:12
1,4021,2,4,20,Operative,2020-03-09 00:00:12
2,32303,8,18,4,Operative,2020-03-09 00:00:12
3,32302,0,7,13,Operative,2020-03-09 00:00:12
4,32301,0,0,17,Close,2020-03-09 00:00:12


In [29]:
df_stations.head()

,station_id,name,latitude,longitude,banking,bonus,bike_stands,postcode
0,4020,Place Edmond Michelet,48.860135,2.350064,1,0,23,75004
1,4021,Centre Georges Pompidou,48.861419,2.352581,1,0,26,75004
2,32303,Gaston Roussel - Commune de Paris,48.894320,2.432552,1,0,30,93230
3,32302,Chemin Vert - Fraternité,48.877141,2.446157,0,0,20,93230
4,32301,Mairie de Romainville,48.885109,2.435679,1,0,17,93230


In [32]:
df_stations[df_stations['postcode'] == "nan"]

,station_id,name,latitude,longitude,banking,bonus,bike_stands,postcode
256,15024,Suffren - La Motte-Piquet,48.850881,2.301220,1,0,23,nan
732,2001,Greneta - Sebastopol,48.865242,2.351667,1,0,30,nan
1103,18108,Evangile - Aubervilliers,48.895040,2.368797,0,0,21,nan
1296,16044,Porte de Saint Cloud - Parc des Princes,48.838663,2.252516,1,0,32,nan
1377,18049,Porte d'Aubervilliers,48.898597,2.368972,0,0,55,nan
1386,21951,Place Centrale,48.787402,2.227883,1,0,22,nan


Save dataframes into .csv files

In [33]:
df_stations.to_csv('df_stations.csv',index=False)

In [34]:
df_activity.to_csv('df_activity.csv',index=False)